<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">Astroquery Beta Test</h1>
  <h2 style="color: #0281c9; font-weight: bold;">ESO Science Archive - April 2025</h2> 
</div>

<hr style="border:2px solid #0281c9"> </hr>

_This notebook focuses specifically on testing recent developments related to the transition from the WDB backend to the TAP backend._ - [DFS-19635](https://jira.eso.org/browse/DFS-19635)

As a Beta tester, we would like to ask your feedback on the following aspects:

 - What absolutely needs to be fixed before the release
 - What would be nice to be fixed before the release
 - Feature ideas and improvements for subsequent releases

The current astroquery ESO module documentation will mostly work: https://astroquery.readthedocs.io/en/latest/eso/eso.html

For the parts that unevitably needed changes, the updated documentation is found in the `docs` folder in this repository:
 - `./docs/AstroqueryEsoDocumentation-Beta.pdf`
 - `./docs/AstroqueryEsoDocstrings-Beta.pdf`
 - `./docs/html/`


In [2]:
!bash install_astroquery.sh

mkdir: cannot create directory ‘TEST_ESO_ASTROQUERY’: File exists
fatal: destination path 'astroquery' already exists and is not an empty directory.
Already on 'TAP'
Your branch is up to date with 'origin/TAP'.
Processing /workspaces/astroquery-beta-test/TEST_ESO_ASTROQUERY/astroquery
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for astroquery: filename=astroquery-0.4.11.dev10181-py3-none-any.whl size=11137130 sha256=bf646d69b7fd4bd795f84dfbdd1848b85d040f26dc0f37d34b9921b97e5fc7ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-j9p46mn5/wheels/b6/28/82/3659dc0de87b199f1911ba27ca92f40086d52605a8cf8e2243
Successfully built astroquery
  Attempting uninstall: astroquery
    Found existing installation: astroquery 0.4.11.dev10181
    Uninstalling astroquery-0.4.11.dev10181:
      Successfully uninstalled astroquery-0.4.11.dev10181

 ##### 

ESO TAP url: https://archive.eso.org/tap_obs

## Handy imports and functions

In [3]:
import os
import pandas as pd
import astropy.table.table as aqtable
from pyvo.dal.exceptions import DALQueryError, DALServiceError
from IPython.display import clear_output

def table_to_csv(aqtable: aqtable, filepath: str):
    aqtable.to_pandas().to_csv(filepath)
    print(f"Table saved to {filepath}")

## ESO instance

In [4]:
from astroquery.eso import Eso

eso_instance = Eso()

eso_instance.maxrec = 1000

print(f"ESO tap_url = {eso_instance._tap_url()}")

ESO tap_url = https://archive.eso.org/tap_obs


# Tutorial

## Query Raw Data

In [5]:
my_table = eso_instance.query_main()
table_to_csv(my_table, "./eso_aq_example_query_main.csv")
# my_table[:2].show_in_notebook()

Table saved to ./eso_aq_example_query_main.csv


/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


## Query Phase 3 Data

In [6]:
available_surveys = eso_instance.list_surveys()

print(pd.DataFrame({"survey": available_surveys}).to_string())

              survey
0         081.C-0827
1         092.A-0472
2         096.B-0054
3        1100.A-0528
4        1101.A-0127
5         193.D-0232
6         195.B-0283
7         196.B-0578
8         196.D-0214
9         197.A-0384
10        198.A-0708
11        60.A-9284H
12         60.A-9493
13             ADHOC
14          ALCOHOLS
15           ALLSMOG
16              ALMA
17             AMAZE
18             AMBRE
19       APEX-SciOps
20          ATLASGAL
21          CAFFEINE
22           ENTROPY
23       ePESSTOplus
24          ERIS-NIX
25     ERIS-SPIFFIER
26          ESPRESSO
27           ESSENCE
28               FDS
29             FEROS
30          Fornax3D
31        FORS2-SPEC
32           GAIAESO
33              GCAV
34           GIRAFFE
35       GOODS_FORS2
36       GOODS_ISAAC
37  GOODS_VIMOS_IMAG
38  GOODS_VIMOS_SPEC
39          GW170817
40             HARPS
41             HAWKI
42              HUGS
43           INSPIRE
44              KIDS
45              KMOS
46           

In [7]:
my_table = eso_instance.query_surveys()

table_to_csv(my_table, "./eso_aq_example_query_surveys.csv")
# my_table[:2].show_in_notebook()

Table saved to ./eso_aq_example_query_surveys.csv


/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [8]:
my_table = eso_instance.query_surveys('SPHERE')

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere.csv")
# my_table.show_in_notebook()

Table saved to ./eso_aq_example_query_surveys_sphere.csv


In [9]:
my_table = eso_instance.query_surveys(['SPHERE', 'VEGAS'])
my_table_2 = eso_instance.query_surveys('SPHERE, VEGAS')
assert all(my_table.values_equal(my_table_2))

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere_vegas.csv")

#my_table.show_in_notebook()

Table saved to ./eso_aq_example_query_surveys_sphere_vegas.csv


## Query Instrument Specific Data

In [10]:
available_instruments = eso_instance.list_instruments()
print(pd.DataFrame({"instrument": available_instruments}).to_string())

         instrument
0            alpaca
1             amber
2              apex
3   apex_quicklooks
4            crires
5             efosc
6              eris
7          espresso
8              fiat
9             fors1
10            fors2
11          giraffe
12          gravity
13            harps
14            hawki
15            isaac
16             kmos
17          matisse
18             midi
19             muse
20             naco
21            nirps
22         omegacam
23          pionier
24          sinfoni
25             sofi
26           sphere
27             uves
28            vimos
29           vircam
30            visir
31            wlgsu
32         xshooter


In [11]:
my_table = eso_instance.query_instrument("midi")
table_to_csv(my_table, "./eso_aq_example_query_instrument_midi.csv")
# my_table.show_in_notebook()

Table saved to ./eso_aq_example_query_instrument_midi.csv


/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


## Query some columns

In [12]:
my_table = eso_instance.query_main(columns=["target", "instrument", "date_obs", "access_url", "datalink_url"])
table_to_csv(my_table, "./eso_aq_example_query_main_columns.csv")
#my_table[0:3].show_in_notebook()

Table saved to ./eso_aq_example_query_main_columns.csv


/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


## Info before launching a query

### Rows and columns

In [13]:
eso_instance.query_main(help=True)

INFO: 
Columns present in the table dbo.raw:
    column_name     datatype    xtype     unit 
------------------- -------- ----------- ------
     access_estsize     long              kbyte
         access_url     char                   
       datalink_url     char                   
           date_obs     char                   
                dec   double                deg
            dec_pnt   double                deg
        det_chip1id     char                   
   det_chop_ncycles    short                   
            det_dit    float                  s
          det_expid    short                   
           det_ndit    short                   
             dp_cat     char                   
              dp_id     char                   
            dp_tech     char                   
            dp_type     char                   
            ecl_lat   double                deg
            ecl_lon   double                deg
          exp_start     char   timestamp   

### Number of records

In [14]:
count_m = eso_instance.query_main(count_only=True)
count_n = eso_instance.query_main(instrument="MUSE", count_only=True)
print(f"Counts: {count_m:0.2e}, {count_n:0.2e}")

count_m = eso_instance.query_surveys('MUSE', count_only=True)
count_n = eso_instance.query_surveys('MUSE', ra=18, dec=0 , radius=1, count_only=True)
print(f"Counts: {count_m:0.2e}, {count_n:0.2e}")

Counts: 3.46e+07, 4.67e+05
Counts: 2.12e+04, 2.20e+01


In [15]:
survey_nrows = {"survey": [], "num_records": []}
for x in available_surveys:
    n = eso_instance.query_surveys(x, count_only=True)
    survey_nrows["num_records"].append(n)
    survey_nrows["survey"].append(x)
    print(x, n)

clear_output()
print(pd.DataFrame(survey_nrows).to_string())

              survey  num_records
0         081.C-0827            1
1         092.A-0472            4
2         096.B-0054            8
3        1100.A-0528            1
4        1101.A-0127            4
5         193.D-0232         5397
6         195.B-0283          587
7         196.B-0578          114
8         196.D-0214          361
9         197.A-0384           28
10        198.A-0708          165
11        60.A-9284H            2
12         60.A-9493            1
13             ADHOC         3720
14          ALCOHOLS            5
15           ALLSMOG           88
16              ALMA       707211
17             AMAZE           25
18             AMBRE            3
19       APEX-SciOps            1
20          ATLASGAL           48
21          CAFFEINE           96
22           ENTROPY            3
23       ePESSTOplus         4472
24          ERIS-NIX          356
25     ERIS-SPIFFIER         1104
26          ESPRESSO        23527
27           ESSENCE          185
28            

### Column names as list

In [16]:
column_list = list(eso_instance.query_main(top=1).columns)
print(pd.DataFrame(column_list).to_string())

                      0
0        access_estsize
1            access_url
2          datalink_url
3              date_obs
4                   dec
5               dec_pnt
6           det_chip1id
7      det_chop_ncycles
8               det_dit
9             det_expid
10             det_ndit
11               dp_cat
12                dp_id
13              dp_tech
14              dp_type
15              ecl_lat
16              ecl_lon
17            exp_start
18             exposure
19          filter_path
20              gal_lat
21              gal_lon
22            grat_path
23            gris_path
24             ins_mode
25           instrument
26           lambda_max
27           lambda_min
28        last_mod_date
29              mjd_obs
30                ob_id
31              ob_name
32               object
33             obs_mode
34             origfile
35               period
36               pi_coi
37              prog_id
38           prog_title
39            prog_type
40              

## Limit the number of records - _top_ vs _maxrec_

 - `top`: "I don't care how many records there are, give me only the _top N_, because my science requires only those."
 - `maxrec`: "Give me as many records as possible given my computational resources and time constraints. Please warn me if results are truncated."

Key Differences:

 - Scope: `TOP` is used within the ADQL query to limit the number of rows returned by that specific query. In contrast, `maxrec` is a TAP service parameter that sets an upper limit on the number of records any query can return, acting as a server-side `safeguard` control.

 - Implementation: `TOP` is specified in the query language (ADQL), whereas `maxrec` is a parameter of the TAP service protocol, often set outside the query itself.

In [17]:
eso_instance.maxrec = None 
eso_instance.query_main(top=3)

access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kbyte,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,nm,nm,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float64,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
25918,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:55:05.270,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:55:05.270,2022-12-07T00:55:05.270,-24.74604694,-24.746047,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:55:05.270,IMAGE,SKY,-30.061165,6.838335,2022-12-07T00:55:05.270Z,120.0,LUMINANCE,-84.089755,190.504407,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03825543982,--,,ALL SKY,s,ALPACA.2022-12-07T00-55-05.270.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,18.87548,18.87548,2022-12-07T00:55:05.270Z,POSITION J2000 18.87548000000001 -24.746047,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25911,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:57:07.400,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:57:07.400,2022-12-07T00:57:07.400,-24.74570194,-24.745702,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:57:07.400,IMAGE,SKY,-30.262454,7.321629,2022-12-07T00:57:07.400Z,120.0,LUMINANCE,-83.666774,192.289978,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03966898148,--,,ALL SKY,s,ALPACA.2022-12-07T00-57-07.400.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.38619194,19.386192,2022-12-07T00:57:07.400Z,POSITION J2000 19.386191999999994 -24.745702,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25775,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:59:14.840,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:59:14.840,2022-12-07T00:59:14.840,-24.74533194,-24.745332,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:59:14.840,IMAGE,SKY,-30.472252,7.827128,2022-12-07T00:59:14.840Z,120.0,LUMINANCE,-83.220412,193.929393,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04114398148,--,,ALL SKY,s,ALPACA.2022-12-07T00-59-14.840.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.91910694,19.919107,2022-12-07T00:59:14.840Z,POSITION J2000 19.919106999999997 -24.745332,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,


In [18]:
eso_instance.maxrec = 3
eso_instance.query_main()

/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kbyte,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,nm,nm,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float64,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
25918,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:55:05.270,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:55:05.270,2022-12-07T00:55:05.270,-24.74604694,-24.746047,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:55:05.270,IMAGE,SKY,-30.061165,6.838335,2022-12-07T00:55:05.270Z,120.0,LUMINANCE,-84.089755,190.504407,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03825543982,--,,ALL SKY,s,ALPACA.2022-12-07T00-55-05.270.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,18.87548,18.87548,2022-12-07T00:55:05.270Z,POSITION J2000 18.87548000000001 -24.746047,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25911,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:57:07.400,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:57:07.400,2022-12-07T00:57:07.400,-24.74570194,-24.745702,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:57:07.400,IMAGE,SKY,-30.262454,7.321629,2022-12-07T00:57:07.400Z,120.0,LUMINANCE,-83.666774,192.289978,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03966898148,--,,ALL SKY,s,ALPACA.2022-12-07T00-57-07.400.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.38619194,19.386192,2022-12-07T00:57:07.400Z,POSITION J2000 19.386191999999994 -24.745702,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25775,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:59:14.840,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:59:14.840,2022-12-07T00:59:14.840,-24.74533194,-24.745332,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:59:14.840,IMAGE,SKY,-30.472252,7.827128,2022-12-07T00:59:14.840Z,120.0,LUMINANCE,-83.220412,193.929393,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04114398148,--,,ALL SKY,s,ALPACA.2022-12-07T00-59-14.840.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.91910694,19.919107,2022-12-07T00:59:14.840Z,POSITION J2000 19.919106999999997 -24.745332,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,


In [19]:
eso_instance.maxrec = 5
eso_instance.query_main(top=10)

access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kbyte,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,nm,nm,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float64,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
25918,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:55:05.270,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:55:05.270,2022-12-07T00:55:05.270,-24.74604694,-24.746047,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:55:05.270,IMAGE,SKY,-30.061165,6.838335,2022-12-07T00:55:05.270Z,120.0,LUMINANCE,-84.089755,190.504407,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03825543982,--,,ALL SKY,s,ALPACA.2022-12-07T00-55-05.270.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,18.87548,18.87548,2022-12-07T00:55:05.270Z,POSITION J2000 18.87548000000001 -24.746047,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25911,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:57:07.400,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:57:07.400,2022-12-07T00:57:07.400,-24.74570194,-24.745702,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:57:07.400,IMAGE,SKY,-30.262454,7.321629,2022-12-07T00:57:07.400Z,120.0,LUMINANCE,-83.666774,192.289978,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03966898148,--,,ALL SKY,s,ALPACA.2022-12-07T00-57-07.400.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.38619194,19.386192,2022-12-07T00:57:07.400Z,POSITION J2000 19.386191999999994 -24.745702,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25775,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:59:14.840,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:59:14.840,2022-12-07T00:59:14.840,-24.74533194,-24.745332,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:59:14.840,IMAGE,SKY,-30.472252,7.827128,2022-12-07T00:59:14.840Z,120.0,LUMINANCE,-83.220412,193.929393,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04114398148,--,,ALL SKY,s,ALPACA.2022-12-07T00-59-14.840.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.91910694,19.919107,2022-12-07T00:59:14.840Z,POSITION J2000 19.919106999999997 -24.745332,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25674,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T01:01:16.900,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T01:01:16.900,2022-12-07T01:01:16.900,-24.74496888,-24.744969,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T01:01:16.900,IMAGE,SKY,-30.672945,8.312446,2022-12-07T01:01:16.900Z,120.0,LUMINANCE,-82.788982,195.322223,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04255671296,--,,ALL SKY,s,ALPACA.2022-12-07T01-01-16.900.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,20.42952305,20.429523,2022-12-07T01:01:16.900Z,POSITION J2000 20.42952299999999 -24.744969,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
26348,https://dataportal.eso

In [20]:
eso_instance.maxrec = 10
eso_instance.query_main(top=5)

access_estsize,access_url,datalink_url,date_obs,dec,dec_pnt,det_chip1id,det_chop_ncycles,det_dit,det_expid,det_ndit,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,filter_path,gal_lat,gal_lon,grat_path,gris_path,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,mjd_obs,ob_id,ob_name,object,obs_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,ra_pnt,release_date,s_region,slit_path,target,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_seqno,tpl_start
kbyte,,,,deg,deg,,,s,,,,,,,deg,deg,,s,,deg,deg,,,,,nm,nm,,d,,,,,,,,,,,deg,deg,,,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,
int64,object,object,object,float64,float64,object,int16,float32,int16,int16,object,object,object,object,float64,float64,object,float32,object,float64,float64,object,object,object,object,float64,float64,object,float64,int32,object,object,object,object,int16,object,object,object,int32,float64,float64,object,object,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,int32,object
25918,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:55:05.270,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:55:05.270,2022-12-07T00:55:05.270,-24.74604694,-24.746047,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:55:05.270,IMAGE,SKY,-30.061165,6.838335,2022-12-07T00:55:05.270Z,120.0,LUMINANCE,-84.089755,190.504407,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03825543982,--,,ALL SKY,s,ALPACA.2022-12-07T00-55-05.270.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,18.87548,18.87548,2022-12-07T00:55:05.270Z,POSITION J2000 18.87548000000001 -24.746047,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25911,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:57:07.400,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:57:07.400,2022-12-07T00:57:07.400,-24.74570194,-24.745702,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:57:07.400,IMAGE,SKY,-30.262454,7.321629,2022-12-07T00:57:07.400Z,120.0,LUMINANCE,-83.666774,192.289978,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.03966898148,--,,ALL SKY,s,ALPACA.2022-12-07T00-57-07.400.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.38619194,19.386192,2022-12-07T00:57:07.400Z,POSITION J2000 19.386191999999994 -24.745702,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25775,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T00:59:14.840,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T00:59:14.840,2022-12-07T00:59:14.840,-24.74533194,-24.745332,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T00:59:14.840,IMAGE,SKY,-30.472252,7.827128,2022-12-07T00:59:14.840Z,120.0,LUMINANCE,-83.220412,193.929393,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04114398148,--,,ALL SKY,s,ALPACA.2022-12-07T00-59-14.840.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,19.91910694,19.919107,2022-12-07T00:59:14.840Z,POSITION J2000 19.919106999999997 -24.745332,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
25674,https://dataportal.eso.org/dataPortal/file/ALPACA.2022-12-07T01:01:16.900,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ALPACA.2022-12-07T01:01:16.900,2022-12-07T01:01:16.900,-24.74496888,-24.744969,CMOS-GS16MP,--,--,--,--,CALIB,ALPACA.2022-12-07T01:01:16.900,IMAGE,SKY,-30.672945,8.312446,2022-12-07T01:01:16.900Z,120.0,LUMINANCE,-82.788982,195.322223,,,,ALPACA,700.0,400.0,2023-06-02T13:20:55.733Z,59920.04255671296,--,,ALL SKY,s,ALPACA.2022-12-07T01-01-16.900.fits,60,ESO PARANAL OBSERVATORY,60.A-9300(B),MASCOT ENGINEERING RUN,0,20.42952305,20.429523,2022-12-07T01:01:16.900Z,POSITION J2000 20.42952299999999 -24.744969,,,1.0,1.0,90.0,--,--,--,--,--,0.0,ALPACA,--,,,--,--,
26348,https://dataportal.eso

In [21]:
eso_instance.maxrec = 1000

## Filter in advance

In [22]:
my_table = eso_instance.query_surveys("sphere, vegas",
                                          columns="obs_collection, calib_level, multi_ob, filter, s_pixel_scale, instrument_name" ,
                                          calib_level=3,
                                          multi_ob='M')

table_to_csv(my_table, "./eso_aq_example_filter_vegas.csv")

Table saved to ./eso_aq_example_filter_vegas.csv


## Filter afterwards

In [23]:
my_table[my_table["s_pixel_scale"] > 0.2]

obs_collection,calib_level,multi_ob,filter,s_pixel_scale,instrument_name
,,,,arcsec,
object,int32,object,object,float64,object
VEGAS,3,M,g_SDSS,0.21,OMEGACAM
VEGAS,3,M,r_SDSS,0.21,OMEGACAM
VEGAS,3,M,g_SDSS,0.2099,OMEGACAM
VEGAS,3,M,r_SDSS,0.2099,OMEGACAM
VEGAS,3,M,u_SDSS,0.2099,OMEGACAM
VEGAS,3,M,g_SDSS,0.2099,OMEGACAM
VEGAS,3,M,u_SDSS,0.2099,OMEGACAM
VEGAS,3,M,r_SDSS,0.2099,OMEGACAM


## Free ADQL query

In [24]:
eso_instance.query_tap_service("Select obs_collection, count(*) from ivoa.ObsCore group by obs_collection order by obs_collection")

obs_collection,COUNT_ALL
object,int32
081.C-0827,1
092.A-0472,4
096.B-0054,8
1100.A-0528,1
1101.A-0127,4
193.D-0232,5397
195.B-0283,587
196.B-0578,114
196.D-0214,361


In [25]:
eso_instance.query_tap_service("Select obs_collection, calib_level, multi_ob, filter, s_pixel_scale, instrument_name "
                               "from ivoa.ObsCore where "
                               "obs_collection in ('sphere', 'vegas') and "
                               "calib_level=3 and "
                               "multi_ob='M' and "
                               "s_pixel_scale < 0.2")

obs_collection,calib_level,multi_ob,filter,s_pixel_scale,instrument_name
,,,,arcsec,
object,int32,object,object,float64,object
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE
SPHERE,3,M,K12,0.0122,SPHERE


## Authenticated queries

 - Even when the login is correct, the query is unauthenticated.
 - No authenticated queries possible --> No risk of data leaks.

In [26]:
eso_instance.login(username='pala')
query_str = "select count(*) from dbo.raw"

INFO: Authenticating pala on 'www.eso.org' ... [astroquery.eso.core]
INFO: Authentication successful! [astroquery.eso.core]


In [27]:
count_unauthenticated = eso_instance.query_tap_service(query_str)
print(count_unauthenticated) # 34531840 in jcarmonaops case

COUNT_ALL
---------
 34602448


In [28]:
# Takes longer - 17s instead of 0.5s
count_authenticated = eso_instance.query_tap_service(query_str, authenticated = True)
print(count_authenticated) 

COUNT_ALL
---------
 34602448


## Print the ADQL string without issuing the query

In [29]:
q_str = eso_instance.query_main(asdasd='asdasd', query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(ra=18, dec=0 , radius=0.1, query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(ra=18, dec=0 , radius=0.1, asdasd='asdasd', query_str_only=True)
print(q_str)

select * from dbo.raw where asdasd = 'asdasd'
select * from dbo.raw where intersects(s_region, circle('ICRS', 18, 0, 0.1))=1
select * from dbo.raw where asdasd = 'asdasd' and intersects(s_region, circle('ICRS', 18, 0, 0.1))=1


## Cone Search

In [30]:
my_table = eso_instance.query_main(ra=12.6521917, dec=-45.5622628, radius=0.05)#, columns="target_name, s_ra, s_dec, s_region")
table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_anna.csv")

Table saved to ./eso_aq_example_cone_search_phase_3_anna.csv


In [31]:
my_table = eso_instance.query_main(ra=189.78, dec=-45.56, radius=0.05, columns="ra, dec, pi_coi")
table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_muse.csv")

#my_table = eso_instance.query_main('XSHOOTER', ra=189.78, dec=-45.56 , radius=0.05)
#table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_muse.csv")


Table saved to ./eso_aq_example_cone_search_phase_3_muse.csv


In [32]:
my_table = eso_instance.query_surveys(ra=150, dec=-59, radius=1, columns="target_name, s_ra, s_dec, s_region")
table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_all.csv")

Table saved to ./eso_aq_example_cone_search_phase_3_all.csv


/home/codespace/.python/current/lib/python3.12/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [33]:
# - Qery survey - # - Phase 3 data

count_m = eso_instance.query_surveys('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print("Phase 3 MUSE: ", count_m)
print()

count_m = eso_instance.query_surveys(ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print("Phase 3 ALL: ", count_m)
print()

# - Query main - # - Raw Data

count_m = eso_instance.query_main('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print("Raw MUSE: ", count_m)
print()

count_m = eso_instance.query_main(ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print("Raw ALL: ", count_m)
print()

# - Query instrument - # - Instrument Specific Raw Data

count_m = eso_instance.query_instrument('MUSE', ra=18, dec=0 , radius=1, count_only=True, query_str_only=False)
print("Instrument Specific Raw - MUSE: ", count_m)
print()

Phase 3 MUSE:  22

Phase 3 ALL:  602

Raw MUSE:  50

Raw ALL:  2245

Instrument Specific Raw - MUSE:  50



## Beta Tester Feedback:

### General free-hand comments and impressions:

...

### Did something break?

 - query_instrument('MUSA') - It broke when I tried to query for the instrument 'MUSA'
 - ...

### Could something be better?

 - query_main() - I don't like the name of this function because...
 - ...


## Features that I'd like the module to support:

Add the features you're like to be implemented. If the feature you want is already listed, please put a star (*) next to it.
 
 - query_catalogues
 - SkyCoord
 - query_region
 - query_object
 - OR, >, <, between, like, ...
 - Uncorrelated random sample queries
 - cutouts - definition and download
 - asyncronous queries
 - ? previews, ssap, datalink
 - parameter `pint_query = True` prints the query executed. Example: `query_main(..., print_query=True)`
 - sort results (order by)

<hr style="border:2px solid #0281c9"> </hr>

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;"> </h1> 
  <h2 style="color: #0281c9; font-weight: bold;"> </h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

In [34]:
# Experiment with your own queries

In [35]:
eso = Eso()
eso.list_instruments()

['alpaca',
 'amber',
 'apex',
 'apex_quicklooks',
 'crires',
 'efosc',
 'eris',
 'espresso',
 'fiat',
 'fors1',
 'fors2',
 'giraffe',
 'gravity',
 'harps',
 'hawki',
 'isaac',
 'kmos',
 'matisse',
 'midi',
 'muse',
 'naco',
 'nirps',
 'omegacam',
 'pionier',
 'sinfoni',
 'sofi',
 'sphere',
 'uves',
 'vimos',
 'vircam',
 'visir',
 'wlgsu',
 'xshooter']

In [36]:
eso.query_instrument('xshooter', help=True)

INFO: 
Columns present in the table ist.xshooter:
      column_name        datatype    xtype       unit  
------------------------ -------- ----------- ---------
          access_estsize     long                 kbyte
              access_url     char                      
            datalink_url     char                      
                date_obs     char                      
                     dec   double                   deg
                 det_dit    float                     s
                det_ndit      int                      
          det_read_clock     char                      
           det_win1_binx    short                      
           det_win1_biny    short                      
           dimm_fwhm_avg    float                arcsec
           dimm_fwhm_rms    float                arcsec
                  dp_cat     char                      
                   dp_id     char                      
                 dp_tech     char                     

In [37]:
table = eso.query_instrument('xshooter', column_filters={'pi_coi': 'PALA/ BREEDT/ GAENSICKE/ SCHMIDTOBREICK/ MARSH/ GREEN/ RODRIGUEZ-GIL/ HERNANDEZ SANTISTEBAN/ ABRIL/ SCHREIBER/ IZQUIERDO SÁNCHEZ/ TOLOZA CASTILLO'}, columns=['ob_targ_name'])
print(table)

ob_targ_name
------------
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
   NSV  6600
         ...
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
ASASSN -13ac
Length = 1000 rows


In [ ]:
table = eso.query_instrument('xshooter', column_filters={'ob_targ_name' : 'V V1129 Cen'}, columns=['ob_targ_name', 'release_date', 'pi_coi']) #, columns=[columns=['ob_targ_name', 'release_date', 'moon_illu'])
print(table)

table1 = table[[any(name.strip().lower() =='Pala' for name in pi.split('/')) for pi in table['pi_coi']]]
#table[['Pala' in pi for pi in table['pi_coi']]]
print(table1)



#print(table["pi_coi"])

#table_headers = eso.get_headers(table['dp_id'])
#table_headers.pprint()

ob_targ_name ...
------------ ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
         ... ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
Length = 69 rows
ob_targ_name release_date pi_coi
------------ ------------ ------
ob_targ_name ...
------------ ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
         ... ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
 V V1129 Cen ...
Length = 69 rows


In [77]:
from astropy.table import conf
conf.max_width = -1   # No limit on table print width
conf.max_lines = -1   # No limit on number of rows printed
conf.max_cols = -1    # No

table = eso.query_instrument('xshooter', column_filters={'ob_targ_name' : 'V V1129 Cen'}, columns=['ob_targ_name', 'release_date', 'pi_coi']) #, columns=[columns=['ob_targ_name', 'release_date', 'moon_illu'])

target_pi = 'Pala'
filtered_table = table[
    [any(name.strip() == target_pi for name in pi.split('/')) for pi in table['pi_coi']]
]
df = filtered_table.to_pandas()
print(df.to_string())  # shows all columns and rows

   ob_targ_name              release_date                                                                                                                                             pi_coi
0   V V1129 Cen  2021-01-03T08:17:27.083Z   Pala/ Belloni/ Breedt/ Gaensicke/ Rodriguez-Gil/ Green/ Hernandez Santisteban/ inight/ Marsh/ Schmidtobreick/ Schreiber/ Toloza Castillo/ Toonen
1   V V1129 Cen  2021-01-03T08:17:27.917Z   Pala/ Belloni/ Breedt/ Gaensicke/ Rodriguez-Gil/ Green/ Hernandez Santisteban/ inight/ Marsh/ Schmidtobreick/ Schreiber/ Toloza Castillo/ Toonen
2   V V1129 Cen  2021-01-03T08:17:28.300Z   Pala/ Belloni/ Breedt/ Gaensicke/ Rodriguez-Gil/ Green/ Hernandez Santisteban/ inight/ Marsh/ Schmidtobreick/ Schreiber/ Toloza Castillo/ Toonen
3   V V1129 Cen  2021-01-03T08:17:56.303Z   Pala/ Belloni/ Breedt/ Gaensicke/ Rodriguez-Gil/ Green/ Hernandez Santisteban/ inight/ Marsh/ Schmidtobreick/ Schreiber/ Toloza Castillo/ Toonen
4   V V1129 Cen  2021-01-03T08:17:57.130Z   Pala/ Bello

In [79]:
my_table = eso_instance.query_main(ra=189.78, dec=-45.56, radius=0.05, columns="ra, dec, pi_coi, instrument")
print(my_table)

     ra         dec    ... instrument
    deg         deg    ...           
------------ --------- ... ----------
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
189.78477888 -45.56407 ...      HARPS
189.78350388  -45.5619 ...       UVES
189.78309111 -45.56234 ...       UVES
189.78333305 -45.56222 ...   XSHOOTER
189.78288611 -45.56218 ...       UVES
   189.78307 -45.56226 ...       UVES
189.78286694 -45.56236 ...   XSHOOTER
         ...       ... ...        ...
189.78286388 -45.56236 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
  189.782685 -45.56218 ...   XSHOOTER
189.78289305 -45.56212 ...   XSHOOTER
Length = 184 rows


In [ ]:
data_files = eso.retrieve_data(table['dp_id'][:2])


INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 2 files ... [astroquery.eso.core]
INFO: Downloading file 1/2 https://dataportal.eso.org/dataPortal/file/XSHOO.2024-04-01T06:48:25.965 to /home/codespace/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Successfully downloaded dataset XSHOO.2024-04-01T06:48:25.965 to /home/codespace/.astropy/cache/astroquery/Eso/XSHOO.2024-04-01T06:48:25.965.fits.Z [astroquery.eso.core]
INFO: Downloading file 2/2 https://dataportal.eso.org/dataPortal/file/XSHOO.2024-04-01T06:48:27.185 to /home/codespace/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Successfully downloaded dataset XSHOO.2024-04-01T06:48:27.185 to /home/codespace/.astropy/cache/astroquery/Eso/XSHOO.2024-04-01T06:48:27.185.fits.Z [astroquery.eso.core]
INFO: Uncompressing file /home/codespace/.astropy/cache/astroquery/Eso/XSHOO.2024-04-01T06:48:25.965.fits.Z [astroquery.eso.core]
INFO: Uncompressing file /home/codespace/.astropy/cache/astroquery/Eso/

In [ ]:
eso.query_instrument('HARPS', help=True)

INFO: 
Columns present in the table ist.HARPS:
    column_name     datatype    xtype     unit 
------------------- -------- ----------- ------
     access_estsize     long              kbyte
         access_url     char                   
       datalink_url     char                   
           date_obs     char                   
                dec   double                deg
      dimm_fwhm_avg    float             arcsec
      dimm_fwhm_rms    float             arcsec
             dp_cat     char                   
              dp_id     char                   
            dp_tech     char                   
            dp_type     char                   
            ecl_lat   double                deg
            ecl_lon   double                deg
          exp_start     char   timestamp       
           exposure    float                  s
            exptime    float                  s
            gal_lat   double                deg
            gal_lon   double             

In [ ]:
table = eso.query_instrument('HARPS', column_filters={'ob_targ_name' : 'V1129_Cen'})#, columns=['ob_targ_name', 'release_date', 'moon_illu'])
print(table)

my_table = eso_instance.query_main(instrument= 'HARPS', ra=12.6521917, dec=-45.5622628, radius=0.05)#, columns="target_name, s_ra, s_dec, s_region")


DALFormatError: str: Error executing the following query:

select * from ist.HARPS where ob_targ_name = 'V1129_Cen'

See examples here: https://archive.eso.org/tap_obs/examples

cause: HTTPSConnectionPool(host='archive.eso.org', port=443): Max retries exceeded with url: /tap_obs/sync (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7e7a6c75c080>, 'Connection to archive.eso.org timed out. (connect timeout=None)'))